In [1]:
import polars as pl

# CVの計算を行う

In [2]:
cv_fold0 = pl.read_csv(
    "../" + "trained_models/e004-add-review-text/valid_dataset_e004-add-review-text.csv"
)
cv_fold1 = pl.read_csv(
    "../"
    + "trained_models/e004-add-review-text-fold1/valid_dataset_e004-add-review-text-fold1.csv"
)
cv_fold2 = pl.read_csv(
    "../"
    + "trained_models/e004-add-review-text-fold2/valid_dataset_e004-add-review-text-fold2.csv"
)

In [3]:
oof = pl.concat([cv_fold0, cv_fold1, cv_fold2])

In [4]:
oof

index,Clothing ID,Age,Title,Review Text,Rating,label,Positive Feedback Count,Division Name,Department Name,Class Name,fold,valid_pred
i64,i64,i64,str,str,i64,i64,i64,str,str,str,i64,f64
3,0,45,"""""","""I was really pleased with this…",5,1,9,"""General""","""Bottoms""","""Skirts""",0,0.998242
5,0,62,"""Beautiful day to evening skirt""","""A lovely skirt and i'm so glad…",4,1,0,"""General""","""Bottoms""","""Skirts""",0,0.99711
6,0,73,"""Too small""","""I usually wear a 6 and ordered…",2,0,7,"""General""","""Bottoms""","""Skirts""",0,0.04778
18,2,38,"""Amazing find!""","""I can?t describe my love for t…",5,1,0,"""General""","""Tops""","""Sweaters""",0,0.9984269
20,2,42,"""Cropped and boxy""","""It is a beautiful sweater, but…",2,0,0,"""General""","""Tops""","""Sweaters""",0,0.1284158
…,…,…,…,…,…,…,…,…,…,…,…,…
9988,232,45,"""Yes!""","""Oh my goodness, i paid full pr…",5,1,0,"""General""","""Dresses""","""Dresses""",2,0.9984633
9989,232,48,"""A colorful big box""","""Since this sold out so fast i …",1,0,0,"""General""","""Dresses""","""Dresses""",2,0.06354
9992,232,52,"""Nicely fitted, lined dress but…","""Great dress, nicely structured…",4,1,0,"""General""","""Dresses""","""Dresses""",2,0.997603


In [6]:
oof.write_csv("../" + "trained_models/e004-0-1-2-ens/xsmall-fold-fix-oof.csv")

In [7]:
from sklearn.metrics import roc_auc_score

cv_score = roc_auc_score(oof["label"], oof["valid_pred"])
print(f"CV Score: {cv_score}")

CV Score: 0.9589851860652018


In [8]:
pred_fold0 = pl.read_csv(
    "../"
    + "trained_models/e004-add-review-text/submission_e004-add-review-text_cv0.9621.csv"
)
pred_fold1 = pl.read_csv(
    "../"
    + "trained_models/e004-add-review-text-fold1/submission_e004-add-review-text-fold1_cv0.9579.csv"
)
pred_fold2 = pl.read_csv(
    "../"
    + "trained_models/e004-add-review-text-fold2/submission_e004-add-review-text-fold2_cv0.9605.csv"
)

In [12]:
((pred_fold0 + pred_fold1 + pred_fold2) / 3).write_csv(
    "../" + f"trained_models/e004-0-1-2-ens/e004-0-1-2-ens-cv{cv_score:.4f}.csv"
)